# Generating and Documenting a Decay Composition

This notebook provides an example workflow for:
1. Reading a TARDIS configuration and model.
2. Generating a new composition by applying nuclear decay.
3. Retrieving and examining decay radiation data such as gamma-ray lines.

It is intended as a reference on how to handle decay processes and gather the resulting radiation data in TARDIS.

In [1]:
# Step 1: Setup and Imports
from pathlib import Path

import astropy.units as u
import pandas as pd

from tardis.energy_input.gamma_ray_transport import get_decay_radiation_data
from tardis.io.atom_data import AtomData
from tardis.io.configuration import config_reader
from tardis.model import SimulationState

%matplotlib inline


/Users/wkerzend/python/tardis/tardis/__init__.py:20: UserWarning: Astropy is already imported externally. Astropy should be imported after TARDIS.
  warnings.warn(


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
# Step 2: Load a Configuration and Atom Data
# Provide a path to a suitable config file:
config_file = Path("../../../tardis/io/configuration/tests/data/tardis_configv1_density_exponential_nebular_Ni_only.yml")  # Example path
config = config_reader.Configuration.from_yaml(config_file)

# Provide a path to your Atom Data file:
atom_data_file = "kurucz_cd23_chianti_H_He.h5"  # Example atom data
atom_data = AtomData.from_hdf(atom_data_file)

# Create a minimal SimulationState for demonstration:
# (In a real workflow, TARDIS will build this more completely.)
model = SimulationState.from_config(config, atom_data=atom_data)


INFO:tardis.io.atom_data.util:
	Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path.
	Exists in TARDIS Data repo /Users/wkerzend/projects/tardis/tardis-data/kurucz_cd23_chianti_H_He.h5
INFO:tardis.io.atom_data.base:Reading Atom Data with: UUID = 6f7b09e887a311e7a06b246e96350010 MD5  = 864f1753714343c41f99cb065710cace 
INFO:tardis.io.atom_data.base:Non provided Atomic Data: synpp_refs, photoionization_data, yg_data, two_photon_data, linelist_atoms, linelist_molecules


## Step 3: Apply Decay to Generate a New Composition
Here we show how to evolve the composition by a certain time interval, reflecting radioactive decay.

In [3]:
model.composition.isotopic_mass_fraction.time_0

<Quantity 0. d>

In [6]:
# Example: Advance the model time by 5 days
time_interval = 10 * u.day

# TARDIS can automatically decay the composition based on the isotopes present.
# Here we show a typical call to decay methods:
decayed_mass_fractions = model.composition.isotopic_mass_fraction.calculate_decayed_mass_fractions(time_interval.to(u.s).value)

# Show the updated isotopic mass fractions:
decayed_mass_fractions.head()

INFO:tardis.model.matter.decay:Decaying abundances for 74649600000.0 seconds


,,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
atomic_number,mass_number,,,,,,,,,,,,,,,,,,,,
26,56,0.479105,0.479105,0.479105,0.479105,0.479105,0.479105,0.479105,0.479105,0.479105,0.479105,0.479105,0.479105,0.479105,0.479105,0.479105,0.479105,0.479105,0.479105,0.479105,0.479105
27,56,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
28,56,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Step 4: Retrieve Decay Radiation Data
The `atom_data` object includes lines and energies emitted during nuclear decay. We examine them here.

In [16]:
# Retrieve decay radiation data:
em_radiation_data, bp_radiation_data = get_decay_radiation_data(
    atom_data.decay_radiation_data, model.composition.isotopic_mass_fraction.index
)

display(em_radiation_data.head())
display(bp_radiation_data.head())




radiation_energy_kev  energy_per_decay_kev  \
atomic_number mass_number                                               
27            56                        511.000            199.290000   
              56                          6.404              0.928580   
              56                          6.391              0.466543   
              56                          7.058              0.121398   
              56                          7.058              0.062110   

                          radiation_type  
atomic_number mass_number                 
27            56                Annihil.  
              56                  XR ka1  
              56                  XR ka2  
              56                  XR kb1  
              56                  XR kb3

radiation_energy_kev  energy_per_decay_kev  \
atomic_number mass_number                                               
27            56                           45.3              0.003715   
              56                           76.7              0.000046   
              56                          178.7              1.860267   
              56                          247.1              0.016556   
              56                          631.2            116.140800   

                          radiation_type  
atomic_number mass_number                 
27            56                     NaN  
              56                     NaN  
              56                     NaN  
              56                     NaN  
              56                     NaN

## Step 5: Inspecting Gamma-Ray Lines
With `gamma_ray_line_dict`, you can explore the line energies and intensities that result from the decaying isotopes in your model.

In [ ]:
# Example preview of gamma-ray lines:
pd.DataFrame.from_dict(gamma_ray_line_dict, orient='index').head()

## Conclusion
This notebook has demonstrated how to:
1. Load a TARDIS model and atom data.
2. Apply radioactive decay to the composition.
3. Retrieve and inspect the decay-related energies and lines.

Use this script as a foundation for your own deeper investigations into nebular or high-energy phenomena in TARDIS!